In [1]:
# we need to change the current working directory to the root of the project
%pwd # show current directory path

'c:\\Code Playground\\Programs_github\\Chicken-Disease-Classification-project\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Code Playground\\Programs_github\\Chicken-Disease-Classification-project'

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig: 
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir: Path


In [25]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories


In [26]:
class ConfigurationManager: 
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ): 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig: 
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config


In [27]:
import os
import urllib.request as request
import zipfile
from cnnClassifier.utils.common import get_size
from cnnClassifier import logger

In [28]:
class DataIngestion: 
    def __init__(self, config: DataIngestionConfig): 
        self.config = config

    def download_file(self): 
        if not os.path.exists(self.config.local_data_file): 
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
            )
            logger.info(f"File downloaded at: {filename} with following headers: {headers}")
        else: 
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        
    
    def extract_zip_file(self): 
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e: 
    raise e

[2023-07-28 17:53:05,980: INFO: common] yaml file: config\config.yaml loaded successfully
[2023-07-28 17:53:05,983: INFO: common] yaml file: params.yaml loaded successfully
[2023-07-28 17:53:05,986: INFO: common] directory created at: artifacts
[2023-07-28 17:53:05,986: INFO: common] directory created at: artifacts/data_ingestion
[2023-07-28 17:53:10,055: INFO: 708910430] File downloaded at: artifacts/data_ingestion/data.zip with following headers: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 624C:25DF85:133660:1AE271:64C3B32D
Accept-Ranges: bytes
Date: Fri, 28 Jul 2023 12:23:10 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830045-CCU
X